# Main Code

In [31]:
import numpy as np
from sklearn.model_selection import train_test_split
from heapq import nlargest
from sklearn.metrics import accuracy_score

from tqdm import tqdm

import time

import torch
import torch.nn as nn
import torch.nn.functional as F
# !pip install torch-optimizer
import torch_optimizer as optim   
from torch.utils.data import Dataset, DataLoader
from torch.nn import MSELoss

In [28]:
import import_ipynb
import MLP
from MLP import MLP_Network

## Get Data

In [3]:
print('loading...', flush = True)
X_train = np.load('./data/X_train.npy', allow_pickle= True)  # X_train and X_test is pickled
Y_train = np.load('./data/Y_train.npy')
X_test = np.load('./data/X_test.npy', allow_pickle= True)
Y_test = np.load('./data/Y_test.npy')
print('done')

loading...
done


## Preprocessing - converting conditions into 'possibility maps'

In [4]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((60000, 2, 8), (60000, 8, 8), (10000, 2, 8), (10000, 8, 8))

In [5]:
def searchcombinationsUtil(k, n):
    """
    k: number of elements (>= 1)
    n: total sum of elements
    return all possible combinations of k numbers that add up to n, regarding its order
    """
    # Recursive function
    
    if k == 1:
        return [[n]]
    else:
        output = []
        for i in range(0, n+1):
            output += [[i]+items for items in searchcombinationsUtil(k-1, n-i)]
        return output        

In [6]:
def pixel_val_calculator(constraint, N):
    """
    constraint: the condition(constraint) of a single row/column in a nonogram
    N: the length of the width/height of the nonogram
    returns a vector with length N, and each value of the vector is the possibility of the corresponding pixel to be colored
    """
    total_colored = np.sum(constraint)

    if(len(constraint) == 0):
        return [0 for _ in range(N)]
    else:
        combinations = searchcombinationsUtil(k=int(len(constraint)+1), n= int(N-total_colored-len(constraint)+1))
        output = []

        for each_combination in combinations:
            pixel_val = []
            for idx, elements in enumerate(each_combination):
                if (idx == 0) or (idx == len(constraint)):
                    pixel_val += [0 for _ in range(elements)]
                else:
                    pixel_val += [0 for _ in range(elements+1)]
                if idx != (len(constraint)):
                    pixel_val += [1 for _ in range(constraint[idx])]
            output.append(pixel_val)
        
        output = np.array(output, dtype = np.float64)
        output = np.sum(output, axis = 0)
        output/= len(combinations)

        return output

In [7]:
pixel_val_calculator([7, 1], 10)

array([0.66666667, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.33333333, 0.33333333, 0.66666667])

In [44]:
def possibility_map_generator(X):
    """
    X: (batch_size, 2, N) or (batch_size, 2, N, t) only if all constraints are composed of same number of blocks (=t)
    (2, N): each nonogram puzzle
    2 stands for each condition (row condition, column condition)
    N stands for the number of pixels (Number of total constraints)
    returns possibility_map: (batch_size, 2, N, N)
    """
    assert X.ndim in [3, 4]
    
    if X.ndim == 3:
        N = X.shape[-1]
    if X.ndim == 4:
        N = X.shape[-2]

    possibility_map = []
    for puzzles in tqdm(X):
        row_condition = puzzles[0]
        row_map = []
        for constraints in row_condition:
            row_map.append(pixel_val_calculator(constraint=constraints, N=N))
        row_map = np.array(row_map)

        column_condition = puzzles[1]
        column_map = []
        for constraints in column_condition:
            column_map.append(pixel_val_calculator(constraint=constraints, N=N))
        column_map = np.array(column_map)
        column_map = column_map.T

        possibility_map.append(np.array([row_map, column_map]))
        
    possibility_map = np.asarray(possibility_map)
    return possibility_map

In [9]:
a = np.array([[[2], [1]], [[1], [2]]])
print(a.shape)
possibility_map_generator(np.expand_dims(a,0))

100%|██████████| 1/1 [00:00<00:00, 964.65it/s]

(2, 2, 1)


array([[[[1. , 1. ],
         [0.5, 0.5]],

        [[0.5, 1. ],
         [0.5, 1. ]]]])

## Neural Network

In [55]:
class NN_Dataset(Dataset):

    def __init__(self, X, X_mapped=None, y=None):

        assert X.ndim in [3, 4]

        if X.ndim == 3:
            N = X.shape[-1]
            self.X = np.array([[np.array([k + [0]*(N - len(k))for k in j]) for j in i] for i in X])
        if X.ndim == 4:
            N = X.shape[-2]


        # pre-calculated possibility map if possible
        if X_mapped is not None:
            self.X_mapped = X_mapped
        else:
            self.X_mapped = possibility_map_generator(X)

        self.y = y
        
    def __getitem__(self, idx):
        inputs = np.array(self.X[idx])
        mapped_inputs = np.array(self.X_mapped[idx], dtype = np.float64)
        

        if self.y is not None:    # Train
            labels = self.y[idx]
            labels = np.array(labels, dtype=np.float64)
            return inputs, mapped_inputs, labels

        else:                     # Test
            return inputs, mapped_inputs
    
    def __len__(self):
        return len(self.X)


In [56]:
def get_loader(batch_size, shuffle, num_workers, X, X_mapped=None, y=None):
    dataset = NN_Dataset(X, X_mapped, y)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
                            num_workers=num_workers)

    print(f'length : {len(dataset)}')
    return data_loader

## Training

In [57]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [90]:
def train(epochs, model, optimizer, train_loader, valid_loader, device):
    start = time.time()

    best_acc = 0
    for epoch in range(1, epochs+1):

        # Train
        model.train()    
        train_accuracies = []
        train_losses = []
        for i, (inputs, mapped_inputs, labels) in enumerate(tqdm(train_loader)):
            
            optimizer.zero_grad()
            mapped_inputs = torch.tensor(mapped_inputs, device=device, dtype=torch.float32)
            labels = torch.tensor(labels, device=device, dtype=torch.float32)
            labels = torch.flatten(labels, start_dim = 1)
            
            preds = model(mapped_inputs)

            loss = MSELoss()
            l = loss(preds, labels)
            
            l.backward()
            optimizer.step()

            preds = preds.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

            for idx, predictions in enumerate(preds):
                total_colored = np.sum(np.array(inputs[idx][0]))
                threshold = np.amin(nlargest(total_colored, predictions))
                predictions = [1 if a >= threshold else 0 for a in predictions]
                acc = accuracy_score(predictions, labels[idx])
                train_accuracies.append(acc)
            
            
            train_losses.append(l.item())
        

        # Validation
        model.eval()
        val_accuracies = []
        val_losses = []
        with torch.no_grad():
            for inputs, mapped_inputs, labels in valid_loader:
                
                mapped_inputs = torch.tensor(mapped_inputs, device=device, dtype=torch.float32)
                labels = torch.tensor(labels, device=device, dtype=torch.float32)
                labels = torch.flatten(labels, start_dim = 1)

                preds = model(mapped_inputs)
                
                val_losses.append(loss.item())

                for idx, predictions in enumerate(preds):
                    total_colored = np.sum(np.array(inputs[idx][0]))
                    threshold = np.amin(nlargest(total_colored, predictions))
                    predictions = [1 if a >= threshold else 0 for a in predictions]
                    acc = accuracy_score(predictions, labels[idx])
                    val_accuracies.append(acc)
            
            val_losses.append(loss.item())

            
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        train_acc = np.mean(train_accuracies)
        val_acc = np.mean(val_accuracies)

        if best_acc < valid_acc:
            best_acc = valid_acc
            best_epoch = epoch
            best_auc = valid_auc
            print('saving model...', flush = True)
            torch.save(model.state_dict(), './weights/MLP.pth')
            print('done')

        print(f'Epoch:{epoch}  Train Loss:{train_loss:.3f} | Valid Loss:{val_loss:.3f}')
        print(f'Train  Acc:{train_acc:.3f}')
        print(f'Valid  Acc:{valid_acc:.3f}')

    end = time.time()
    print(f'\nEpoch Process Time: {(end-start)/60:.2f}Minute')
    print(f'Best Epoch:{best_epoch}, Best Acc:{best_acc:.3f}')

In [76]:
X_train_mapped = possibility_map_generator(X_train)
X_test_mapped = possibility_map_generator(X_test)

X_train, X_val, X_train_mapped, X_val_mapped, Y_train, Y_val = train_test_split(X_train, X_train_mapped, Y_train, test_size = 0.2, random_state = 42) 

100%|██████████| 10000/10000 [00:47<00:00, 211.23it/s]


In [91]:
print(X_train.shape, X_val.shape, X_train_mapped.shape, X_val_mapped.shape, Y_train.shape, Y_val.shape)

(38400, 2, 8) (9600, 2, 8) (38400, 2, 8, 8) (9600, 2, 8, 8) (38400, 8, 8) (9600, 8, 8)


In [92]:
train_loader = get_loader(batch_size=32, shuffle=True, num_workers=0, X = X_train, X_mapped=X_train_mapped, y = Y_train) 
valid_loader = get_loader(batch_size=32, shuffle=False, num_workers=0, X = X_val, X_mapped = X_val_mapped, y = Y_val) 
epochs = 100
lr = 0.001

num_pixels = X_train.shape[-2] if X_train.ndim is 4 else X_train.shape[-1]

model = MLP_Network(num_pixels = num_pixels).to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr)

length : 38400
length : 9600


In [93]:
best_epoch, best_auprc, best_auc = train(epochs = epochs, model = model, optimizer = optimizer, train_loader = train_loader, \
                                         valid_loader = valid_loader, device = device)

  0%|          | 0/1200 [00:00<?, ?it/s]C:\Users\qw070\anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\qw070\anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
  0%|          | 2/1200 [00:00<01:02, 19.25it/s]

torch.Size([32, 64]) torch.Size([32, 64])
torch.Size([32, 64]) torch.Size([32, 64])
torch.Size([32, 64]) torch.Size([32, 64])
torch.Size([32, 64]) torch.Size([32, 64])


  0%|          | 5/1200 [00:00<01:06, 18.06it/s]

torch.Size([32, 64]) torch.Size([32, 64])
torch.Size([32, 64]) torch.Size([32, 64])
torch.Size([32, 64]) torch.Size([32, 64])


  1%|          | 9/1200 [00:00<01:09, 17.07it/s]

...

torch.Size([32, 64]) torch.Size([32, 64])
torch.Size([32, 64]) torch.Size([32, 64])
torch.Size([32, 64]) torch.Size([32, 64])


 18%|█▊        | 221/1200 [00:11<02:05,  7.83it/s]

torch.Size([32, 64]) torch.Size([32, 64])


 18%|█▊        | 221/1200 [00:12<00:54, 17.98it/s]


KeyboardInterrupt: 

## Inference

In [ ]:
def inference(model, test_loader, device):
    pred_list = []
    label_list = []

    model.eval()
    with torch.no_grad():
        for inputs in test_loader:
            
            inputs = torch.unsqueeze(inputs, 1)
            inputs = torch.tensor(inputs, device=device, dtype=torch.float32)
            
            preds = model(inputs)
            preds = preds.view(preds.nelement())
            pred_list += (list(preds.detach().cpu().numpy()))

        pred_list = np.array(pred_list)
        
    return pred_list

In [ ]:
model = Network(init_channel = 1)
model.load_state_dict(torch.load('./non_cardiac_weights/CNN_single_model.pth'))
model = model.cuda()   # if cuda is available

In [ ]:
test_loader = get_loader(batch_size=256, shuffle=False, num_workers=6, X = x_test, y = None) 
CNN_single_prediction = inference(model = model, test_loader = test_loader, device = device)